<a href="https://colab.research.google.com/github/maricari/NLP/blob/main/3c_Custom_embedding_con_Gensim_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



## Alumna: María Carina Roldán

### Objetivo
Utilizar un corpus para crear embeddings de palabras basado en ese contexto, y luego explicar los resultados.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Se trabajará con dos fuentes de datos.
* Una página con refranes alusivos al tiempo. (la misma utilizada para el boy con Spacy)
* Una página con frases positivas, refranes y dichos (un poco graciosas o cursis :P pero útiles para el ejercicio)

In [2]:
import numpy as np

# Para leer y parsear el texto en HTML
import urllib.request
import bs4 as bs

# Para limpiar el texto
import re

#### Lee y parsea la página de refranes alusivos al tiempo

In [3]:
raw_html = urllib.request.urlopen('https://www.cervantesvirtual.com/obra-visor/refranes-alusivos-al-tiempo/html/')
raw_html = raw_html.read()

# Parsea el artículo
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Extrae el texto
# Disclaimer!
# Este texto es ad-hoc para esta página. No servirá para una página genérica

article_section = article_html.find('section')
article_text = article_section.text.split('PANIZO RODRIGUEZ, Juliana\n')[1].split('BIBLIOGRAFÍA')[0]
for titulo in ('EL TIEMPO','EL DIA','LOS DIAS DE LA SEMANA','EL SOL','EL CALOR','EL FRIO','LA HELADA','EL AGUA','EL VIENTO','LA PIEDRA, LOS TRUENOS Y LA NIEBLA','LA NOCHE'):
  article_text = article_text.replace(titulo, '')

# Remueve espacios, saltos de línea o tabulación que pudieran haber quedado
text = re.sub(r'\s+', ' ', article_text)
text

'Para Rodríguez Marín el refrán es "un dicho popular, sentencioso y breve, de verdad comprobada, generalmente simbólico y expuesto en forma poética, que contiene una regla de conducta u otra enseñanza".Los refranes que inserto a continuación alusivos al tiempo, algunos los he recopilado en Valladolid y pueblos de la provincia y otros proceden de las obras señaladas en la bibliografía. La edad de los informantes oscila entre los veintidós y los ochenta y ocho años.Señalan, entre otros, los siguientes aspectos:-El valor del tiempo: El tiempo es oro. El tiempo es la cosa más preciosa del mundo. El tiempo, que es lo que más vale, no lo da Dios de balde. Quien defiende su tiempo, defiende su dinero.-Los efectos curativos del tiempo: El tiempo cura más que el sol. El tiempo es gran médico para el alma y para el cuerpo. El tiempo todo lo cura y todo lo muda. No hay mal que el tiempo no alivie su tormento.-La invitación a aprovechar el tiempo: Aprovecha el tiempo, que vale el cielo. Con el tie

In [4]:
corpus1 = text.split(".") # divide en oraciones
corpus1[116:120]

['Tiempo pasado, con pena recordado',
 'Tiempo perdido, para siempre ido',
 'Tiempo mal gastado, nunca recobrado',
 'Tiempo no aprovechado, viento que ha pasado']

In [5]:
print(f"Cantidad de documentos: {len(corpus1)}")

Cantidad de documentos: 290


#### Lee y parsea la página de frases positivas, refranes y dichos

In [6]:
raw_html = urllib.request.urlopen('https://www.xuliocs.com/frasespositiv.htm')
raw_html = raw_html.read()

# Parsea el artículo
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')
# text = re.sub(r'\s+', ' ', article_text)

article_text = ''
for para in article_paragraphs:
    text = re.sub(r'\(.+\)', ' ', para.text) # elimino los autores
    article_text += text
# clean del texto (muy básico! puede mejorarse)
text = article_text.replace('\n',"").replace('""','').replace('"','.').replace('”','.')
text = re.sub(r'[ ]{2,}', ' ', text)
text = text.replace('. .','.')
text = re.sub(r'\.+', '.', text)
text

"\xa0* Inicio: ENTRADA* El autor: Xulio* Fotos de esta pÃ¡xina web*Los pueblos del conceyu Lena Algunas frasesa) .Son las nuevas armas de la libertad: bolígrafos, ordenadores, radios, cámaras. b) Con la herencia de Stephen Hawking:“Uno: recuerda mirar hacia arriba, hacia las estrellas, y no a tus pies. Dos: nunca dejes el trabajo, Ã©l te darÃ¡ un significado y un propÃ³sito en la vida sin el cual tu existencia estarÃ¡ vacÃ\xada. Tres: si tienes la suerte de encontrar el amor, recuerda que es algo muy raro, no lo tires nunca por la borda.c) O porque tal vez también.Beber del pasado permite al grupo humano encontrar la energía para afrontar el presente y preparar su futuro. ,d) Y como decÃ\xada Salvador Allende: porque.La historia es nuestra, y la hacen los pueblos.e) O como dice Eduardo Galeano: .Al fin y al cabo, somos lo que hacemos, para cambiar lo que somos.f) Y otras tantas frases para los tiempos.g) Pues, en fin, con las palabras bastante más musicales de Raimon,.El que lo sepa to

In [7]:
corpus2 = text.split(".") # divide en oraciones
corpus2[230:233]

['Tras de hoy, vien mañana',
 'Tras la tempestad, siempre viene la calma',
 'Tu fortaleza podría ser tu debilidad']

In [8]:
print(f"Cantidad de documentos: {len(corpus2)}")

Cantidad de documentos: 268


### 1 - Preprocesamiento

In [9]:
from keras.preprocessing.text import text_to_word_sequence

In [10]:
# Texto 1
sentence_tokens1 = []
# Recorre el corpus y transforma las oraciones
# en una secuencia de palabras
for _, row in enumerate(corpus1):
    sentence_tokens1.append(text_to_word_sequence(row))

# sample
sentence_tokens1[116:120]


[['tiempo', 'pasado', 'con', 'pena', 'recordado'],
 ['tiempo', 'perdido', 'para', 'siempre', 'ido'],
 ['tiempo', 'mal', 'gastado', 'nunca', 'recobrado'],
 ['tiempo', 'no', 'aprovechado', 'viento', 'que', 'ha', 'pasado']]

In [11]:
# Texto 2
sentence_tokens2 = []
# Recorre el corpus y transforma las oraciones
# en una secuencia de palabras
for _, row in enumerate(corpus2):
    sentence_tokens2.append(text_to_word_sequence(row))

# sample
sentence_tokens2[230:233]

[['tras', 'de', 'hoy', 'vien', 'mañana'],
 ['tras', 'la', 'tempestad', 'siempre', 'viene', 'la', 'calma'],
 ['tu', 'fortaleza', 'podría', 'ser', 'tu', 'debilidad']]

### 2 - Creación de vectores y entrenamiento

#### Función auxiliar para la creación de vectores (word2vec)

In [12]:
from gensim.models.callbacks import CallbackAny2Vec
# Sobracargamos el callback de gensim para poder ver el "loss"
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

#### 2.1 - Modelo para el texto 1

In [13]:
w2v_model1 = Word2Vec(min_count=7,   # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=52,        # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram


In [14]:
# Buildear el vocabulario con los tokens
w2v_model1.build_vocab(sentence_tokens1)
print(f"Corpus\nCantidad de docs: {w2v_model1.corpus_count}\nCantidad de words distintas {len(w2v_model1.wv.vocab)}")

Corpus
Cantidad de docs: 290
Cantidad de words distintas 96


In [15]:
# Entrenamos el modelo generador de vectores
w2v_model1.train(sentence_tokens1,
                 total_examples=w2v_model1.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 24171.73046875
Loss after epoch 1: 9059.984375
Loss after epoch 2: 9140.9296875
Loss after epoch 3: 8843.51953125
Loss after epoch 4: 8885.43359375
Loss after epoch 5: 9200.72265625
Loss after epoch 6: 8861.3125
Loss after epoch 7: 8510.09375
Loss after epoch 8: 9015.140625
Loss after epoch 9: 8485.4140625
Loss after epoch 10: 8566.25
Loss after epoch 11: 8697.265625
Loss after epoch 12: 8164.609375
Loss after epoch 13: 8285.875
Loss after epoch 14: 8542.296875
Loss after epoch 15: 8362.515625
Loss after epoch 16: 8787.8125
Loss after epoch 17: 8085.265625
Loss after epoch 18: 8868.9375
Loss after epoch 19: 8834.40625
Loss after epoch 20: 8386.78125
Loss after epoch 21: 9029.75
Loss after epoch 22: 8854.984375
Loss after epoch 23: 8084.71875
Loss after epoch 24: 8554.15625
Loss after epoch 25: 8311.078125
Loss after epoch 26: 8802.09375
Loss after epoch 27: 8494.890625
Loss after epoch 28: 8513.25
Loss after epoch 29: 8375.90625
Loss after epoch 30: 8304.8125
Loss a

(53623, 239100)

#### 2.2 - Modelo para el texto 2

In [16]:
w2v_model2 = Word2Vec(min_count=4,   # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=52,        # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram


In [17]:
# Buildear el vocabulario con los tokens
w2v_model2.build_vocab(sentence_tokens2)
print(f"Corpus\nCantidad de docs: {w2v_model2.corpus_count}\nCantidad de words distintas {len(w2v_model2.wv.vocab)}")

Corpus
Cantidad de docs: 268
Cantidad de words distintas 104


In [18]:
# Entrenamos el modelo generador de vectores
w2v_model2.train(sentence_tokens2,
                 total_examples=w2v_model2.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 14416.96875
Loss after epoch 1: 11922.53125
Loss after epoch 2: 5444.001953125
Loss after epoch 3: 4226.537109375
Loss after epoch 4: 4184.50390625
Loss after epoch 5: 3935.54296875
Loss after epoch 6: 3934.71875
Loss after epoch 7: 3950.8671875
Loss after epoch 8: 4048.3984375
Loss after epoch 9: 3835.00390625
Loss after epoch 10: 4223.98828125
Loss after epoch 11: 3826.8359375
Loss after epoch 12: 4119.2265625
Loss after epoch 13: 4099.1796875
Loss after epoch 14: 3828.7578125
Loss after epoch 15: 4194.65625
Loss after epoch 16: 3977.4609375
Loss after epoch 17: 4111.5703125
Loss after epoch 18: 3996.6953125
Loss after epoch 19: 3826.03125
Loss after epoch 20: 3673.1484375
Loss after epoch 21: 4028.2890625
Loss after epoch 22: 4016.90625
Loss after epoch 23: 4525.5546875
Loss after epoch 24: 3982.859375
Loss after epoch 25: 3988.96875
Loss after epoch 26: 3758.359375
Loss after epoch 27: 4082.03125
Loss after epoch 28: 3866.8125
Loss after epoch 29: 4098.140625
Lo

(58528, 274900)

### 3 - Visualización de agrupaciones de vectores

In [19]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import warnings
warnings.filterwarnings('ignore')

import plotly.graph_objects as go
import plotly.express as px


In [20]:
def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

#### 3.1 - Visualización de embeddings del texto 1

In [21]:
# Grafica los embeddings del texto1 en 2D
x_vals, y_vals, labels = reduce_dimensions(w2v_model1)

MAX_WORDS=300

fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS],  width=1000, height=1000)
fig.update_layout( xaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1),
                   yaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1))
fig.show(renderer="colab")

#### Observaciones (texto 1)
- Los días de la semana aparece agrupados (lunes, martes, jueves, etc.)
- Hay algunos pares de palabras que aparecen próximas entre sí y esto semánticamente tiene sentido, como mes y año, días y semana, lluvia y heladas (aunque estas dos, alejadas de llueve).
- En general no se ven "clusters" de palabras y sí muchas palabras sin utilidad como artículos o preposiciones.
- A pesar de haber ajustado el parámetro min_count de Word2Vec para mejorar la relación entre la cantidad de documentos y las palabras distintas, el corpus no parece ser suficiente para la construcción de embeddings de interés.



#### 3.2 - Visualización de embeddings del texto 2

In [22]:
# Grafica los embeddings del texto2 en 2D
x_vals, y_vals, labels = reduce_dimensions(w2v_model2)

MAX_WORDS=300
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS], width=1100, height=1000)
fig.update_layout( xaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1),
                   yaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1))
fig.show(renderer="colab")

#### Observaciones (texto 2)
- Aunque el texto tiene cantidad de documentos y palabras distintas similares a los del texto 1, en este caso las palabras se veían más dispersas aún, por lo que decidí subir el parámetro min_count de Word2Vec y los resultados mejoraron un poco.
- En este caso se ven varias palabras "superpuestas", es decir con vectores muy muy cercanos. Es el caso de *más* y *menos*, que aunque tengan significado contrario, parece que aparecen mucho en el mismo contexto. Lo validamos con el corpus:
  - *Cuanto menos se tiene, más contento se está*
  - *Él siempre agarraba la más grande y menos valiosa*
- Otro ejemplo de palabras muy próximas son *nadie* y *puede*. Validamos con el corpus y encontramos una frase donde la combinación se repite: "*Porque nadie puede saber por ti. Nadie puede crecer por ti. Nadie puede buscar por ti. Nadie puede hacer por ti lo que tú mismo debes hacer*". Al ser un corpus medianamente chico, solo esta frase pesa un montón en los resultados.


### 4 - Algunos términos de interés

In [23]:
# Palabras que más se relacionan con 'martes'
w2v_model1.wv.most_similar(positive=["martes"], topn=10)


[('lunes', 0.9895482063293457),
 ('jueves', 0.9642949104309082),
 ('viernes', 0.9605574607849121),
 ('hay', 0.959932804107666),
 ('tu', 0.9529927968978882),
 ('sábado', 0.9438839554786682),
 ('ni', 0.9433125853538513),
 ('le', 0.9320670962333679),
 ('sin', 0.9304990768432617),
 ('en', 0.9146996736526489)]

In [38]:
w2v_model2.wv.most_similar(positive=["más"])

[('menos', 0.975990891456604),
 ('es', 0.9620974063873291),
 ('para', 0.9505943655967712),
 ('mejor', 0.9412703514099121),
 ('siempre', 0.9323410391807556),
 ('hay', 0.9318129420280457),
 ('la', 0.928756833076477),
 ('sino', 0.9121649265289307),
 ('mucho', 0.9120205640792847),
 ('otra', 0.9092636704444885)]

In [24]:
# Palabras que más se relacionan con 'día' en ambos corpus
w2v_model1.wv.most_similar(positive=["día"])

[('viento', 0.9924243688583374),
 ('al', 0.9922174215316772),
 ('suele', 0.9905498027801514),
 ('por', 0.9887723922729492),
 ('como', 0.9883795976638794),
 ('mes', 0.9882107973098755),
 ('noche', 0.9866777658462524),
 ('invierno', 0.9860680103302002),
 ('viene', 0.9857591390609741),
 ('está', 0.9849348068237305)]

In [25]:
w2v_model2.wv.most_similar(positive=["día"])

[('un', 0.9715929627418518),
 ('vale', 0.9645786881446838),
 ('una', 0.9592303037643433),
 ('soy', 0.9575828909873962),
 ('sueño', 0.9569344520568848),
 ('vida', 0.9472383260726929),
 ('otra', 0.9469289779663086),
 ('mi', 0.9423478841781616),
 ('sólo', 0.9312440156936646),
 ('le', 0.9290863275527954)]

In [26]:
# Palabras que más se relacionan con 'tiempo' en ambos corpus
w2v_model1.wv.most_similar(positive=["tiempo"])

[('con', 0.9907104969024658),
 ('vale', 0.9902099370956421),
 ('todo', 0.9884042739868164),
 ('es', 0.9879158139228821),
 ('cura', 0.9878525733947754),
 ('paciencia', 0.9873161315917969),
 ('lo', 0.9857999086380005),
 ('ha', 0.9843674302101135),
 ('mucho', 0.983360767364502),
 ('y', 0.9803305268287659)]

In [27]:
w2v_model2.wv.most_similar(positive=["tiempo"])

[('si', 0.9581272602081299),
 ('hombre', 0.9562689661979675),
 ('ser', 0.9540499448776245),
 ('muy', 0.9526740312576294),
 ('mañana', 0.9473334550857544),
 ('solo', 0.9434913396835327),
 ('le', 0.9365193843841553),
 ('mucho', 0.935633659362793),
 ('estar', 0.9185158014297485),
 ('nunca', 0.9158368110656738)]

In [28]:
# Palabras que menos se relacionan con "personas"
w2v_model1.wv.most_similar(negative=["personas"], topn=10)

[('sin', -0.5904620885848999),
 ('ni', -0.6104201078414917),
 ('tu', -0.6658000946044922),
 ('sábado', -0.6925266981124878),
 ('martes', -0.7337359189987183),
 ('lunes', -0.7418662905693054),
 ('viernes', -0.7452782392501831),
 ('jueves', -0.7994962930679321),
 ('manifiesto', -0.8532463312149048),
 ('sol', -0.8550961017608643)]

In [29]:
# Palabras que se relacionan positicvamente con "día"pero negativamente con "personas"
w2v_model1.wv.most_similar(positive=["día"], negative=["personas"])

[('tu', 0.19913801550865173),
 ('san', 0.1867763102054596),
 ('si', 0.18540479242801666),
 ('ni', 0.16825293004512787),
 ('año', 0.15131333470344543),
 ('buen', 0.14701610803604126),
 ('viernes', 0.14677725732326508),
 ('sin', 0.14466805756092072),
 ('agua', 0.14348426461219788),
 ('porque', 0.14025545120239258)]

### Conclusiones Similitudes y Diferencias
- Mirando algunas palabras en forma individual, las similaridades tienen sentido. Solo usando el parámetro 'positive' los resultados coinciden con la proximidad de los vectores en el gráfico, y a veces (no siempre) con la cercanía del significado de las palabras.
- La similitud con parámetro 'negative' es más difícil de interpretar. 
Mirando el código fuente de gensim (https://github.com/RaRe-Technologies/gensim/blob/develop/gensim/models/keyedvectors.py#L776) pude entender desde el punto de vista matemático el significado de combinar positive y negative. El algoritmo hace un vector "promedio" dando a los vectores originales pesos positivos o negativos según corresponda, y luego busca los términos similares a este nuevo vector. Desde esta teoría y mirando los valores en los gráficos, el resultado es interpretable. Desde el punto de vista semántico o de contexto dentro del corpus, es más complicado. 



### 5 - Combinamos los dos corpus

In [30]:
sentence_tokens3 = sentence_tokens1 + sentence_tokens2

In [31]:
w2v_model3 = Word2Vec(min_count=5,   # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=52,        # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram


In [32]:
# Buildear el vocabulario con los tokens
w2v_model3.build_vocab(sentence_tokens3)
print(f"Corpus\nCantidad de docs: {w2v_model3.corpus_count}\nCantidad de words distintas {len(w2v_model3.wv.vocab)}")

Corpus
Cantidad de docs: 558
Cantidad de words distintas 207


In [33]:
# Entrenamos el modelo generador de vectores
w2v_model3.train(sentence_tokens3,
                 total_examples=w2v_model3.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 50361.23828125
Loss after epoch 1: 22267.28515625
Loss after epoch 2: 21445.484375
Loss after epoch 3: 20329.1484375
Loss after epoch 4: 20431.828125
Loss after epoch 5: 21089.625
Loss after epoch 6: 20799.25
Loss after epoch 7: 20934.375
Loss after epoch 8: 20365.421875
Loss after epoch 9: 20576.65625
Loss after epoch 10: 20400.359375
Loss after epoch 11: 20537.078125
Loss after epoch 12: 20635.78125
Loss after epoch 13: 21079.4375
Loss after epoch 14: 20529.0
Loss after epoch 15: 20128.0
Loss after epoch 16: 20919.25
Loss after epoch 17: 20597.4375
Loss after epoch 18: 20146.9375
Loss after epoch 19: 20661.53125
Loss after epoch 20: 20431.96875
Loss after epoch 21: 20589.25
Loss after epoch 22: 19902.4375
Loss after epoch 23: 19974.21875
Loss after epoch 24: 19683.875
Loss after epoch 25: 19873.0625
Loss after epoch 26: 19536.8125
Loss after epoch 27: 19132.3125
Loss after epoch 28: 19176.9375
Loss after epoch 29: 19276.0
Loss after epoch 30: 18684.625
Loss after 

(237044, 753100)

In [34]:
# Grafica los embeddings del texto3 en 2D
x_vals, y_vals, labels = reduce_dimensions(w2v_model3)

MAX_WORDS=300
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS], width=1100, height=1000)
fig.update_layout( xaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1),
                   yaxis = dict(tickmode = 'linear', tick0 = 0,dtick = 1))
fig.show(renderer="colab")

Exploramos la similitud de algunas palabras

In [35]:
w2v_model3.wv.most_similar(positive=["día"])

[('siguiente', 0.7196047306060791),
 ('fiesta', 0.7063302993774414),
 ('noche', 0.6262022852897644),
 ('mañana', 0.6200934648513794),
 ('agosto', 0.5915057063102722),
 ('mes', 0.5730820894241333),
 ('primer', 0.5671219825744629),
 ('julio', 0.5645145177841187),
 ('trabajo', 0.5559523701667786),
 ('sueño', 0.5389246344566345)]

In [36]:
w2v_model3.wv.most_similar(positive=["tiempo"])

[('el', 0.6532574892044067),
 ('paso', 0.6488692164421082),
 ('muda', 0.6277015209197998),
 ('oro', 0.6194952726364136),
 ('con', 0.5921300649642944),
 ('ido', 0.582643985748291),
 ('perdido', 0.566999077796936),
 ('jamás', 0.5666837692260742),
 ('cosas', 0.542362630367279),
 ('vuela', 0.5256019830703735)]

In [37]:
w2v_model3.wv.most_similar(positive=["lluvia"])

[('calor', 0.7326878309249878),
 ('tres', 0.7022217512130737),
 ('julio', 0.6756519079208374),
 ('mayo', 0.6742298603057861),
 ('enero', 0.6695873737335205),
 ('heladas', 0.669151782989502),
 ('campo', 0.6645703315734863),
 ('agua', 0.6615532636642456),
 ('veces', 0.6386480927467346),
 ('virgen', 0.6317189931869507)]

### Conclusiones (texto unificado)
Al unificar los dos corpus, los cuales tienen algún vocabulario en común) se puede ver que el incremento de la cantidad de información (cantidad de documentos y cantidad de palabras diferentes) se obtienen mejores respuestas:
- aparecen juntos los meses - enero, febrero, julio y agosto - y alguna información relativa al clima - lluvia, helada, agua, calor, llover
- mirando palabras en forma individual (en particular las mismas que habíamos explorado por separado en los textos 1 y 2) podemos ver que los resultados devuelven palabras con mucha relación desde el punto de vista semántico.

En conclusión, trabajar con un corpus mayor dio resultados más interesantes.